In [13]:
import numpy as np
# CONSTANTS
n = 1          
g = 20        # Degrees of freedom
c1 = np.sqrt(4*np.pi**3/45)
c2 = 2*np.pi**2/45
DecayConstant = 1

In [8]:
def g_star(Temp):
    # Find closest temperature value in Gstardata
    closest_temp = min(Gstardata.keys(), key=lambda t: abs(t - Temp))
    return Gstardata[closest_temp]

def g_starS(Temp):
    # Find closest temperature value in GstarSdata
    closest_temp = min(GstarSdata.keys(), key=lambda t: abs(t - Temp))
    return GstarSdata[closest_temp]

In [10]:
def Y_EQ(x):
    return 0.145 * (g / g_starS(m / x)) * x**(3/2) * np.exp(-x) #eqn 5.25

# Define H(m) function (Hubble parameter)
def H_m(x):
    return (c1 * np.sqrt(g_star(m / x)) * (m / x)**2) / M_Pl

# Define s(x, m) function (entropy density)
def s(x, m):
    return c2 * g_star(m / x) * (m / x)**3

In [26]:
def Energy(Velocity, Mass): #E = sqrt(m^2 + p^2)
    P = 0.5*Mass*(Velocity**2)
    return np.sqrt(Mass**2 + P**2) #energy will change with the temperature. the cross section is (1/2s)(amplitude)

# Cross-section formula for a single Lambda
def sigma(s, *lambdas, particle=None):
    if particle is not None:
        lambdas = particle
    return sum(Lambda / (2 * s) for Lambda in lambdas) #Make an array that has lambdas_pion, lambdas_proton, lambdas_neutron, etc. for all of their interactions
    
def thermally_averaged_cross_section(T, m, Lambda):
    def integrand(s):
        return sigma(s, Lambda) * np.sqrt(s) * kn(1, np.sqrt(s) / T)
        
    integral_result, error = quad(integrand, 4 * m**2, np.inf)
    return (1 / (8 * m**4 * T * kn(2, m / T))) * integral_result

# Lambda_x function for freeze-out dynamics
def lambda_x(x, m, g_s, g_star, Lambda):
    H_m_value = H_m(x)  
    T = m / x
    thermally_avg_sigma = thermally_averaged_cross_section(T, m, Lambda)
    s_value = s(x, m)
    return (x * thermally_avg_sigma * s_value) / H_m_value # Equation 1.57 in my thesis notes.

# Differential equation for relic abundance
def dYdx(x, Y, Lambda, particle):
    Y_eq = Y_EQ(x)
    ratio = particle_dict.get(particle, 1)  # Get the particle's ratio; default to 1 if not found
    lambda_val = lambda_x(x * ratio, m, g_starS, g_star, Lambda)

    if x > 100:  # Arbitrary cutoff for equilibrium
        return np.array([0])
    result = -lambda_val * x**(-n-2) * (Y**2 - Y_eq**2)
    return np.array([result])

def get_variable_name(var, global_vars):
    names = [name for name, value in global_vars.items() if value is var]
    return names[0] if names else "unknown"

In [20]:
def AvgDecayRate(x):
    return np.exp(-DecayConstant*x) #Ae^-lambda*t but using x instead

def CoupleddYdx(x, Y, Y_prod, AvgDecayRate, Lambda):
    Y_eq = Y_EQ(x)  
    lambda_val = lambda_x(x, m, g_starS, g_star, Lambda)
    AvgDecayRate_val = AvgDecayRate(x)
    
    if x > 100:  # Arbitrary cutoff for equilibrium; modify as needed
        return np.array([0])
    #NOTE THAT YOU NEED TO DEFINE A DECAY RATE, Y_PROD AND Y_PROD_EQ
    result = -lambda_val * x**(-n-2) * (Y**2 - Y_eq**2) - ((1.661*x)/((m**2)*(g_star**1/2))*AvgDecayRate_val*(Y-Y_eq*(Y_prod/Y_prod_eq)**2)) 
    return np.array([result])

def dYproddx(x, Y, Y_prod, AvgDecayRate, Lambda):
    AvgDecayRate_val = AvgDecayRate(x)
    
    if x > 100:  # Arbitrary cutoff for equilibrium; modify as needed
        return np.array([0])
        
    result = - ((1.661*x)/((m**2)*(g_star**1/2))*AvgDecayRate_val*(Y-Y_eq*(Y_prod/Y_prod_eq)**2))
    return np.array([result])

# To do list
##### Want sum(m_P * Y) = 4e-10 GeV so all the particle masses multiplied by the relic abundance should be equal to 4e-10 GeV

In [ ]:
#def sigma(s, Lambda):
#    return Lambda / (2 * s)

"""
# Differential equation for relic abundance
def dYdx(x, Y, Lambda):
    Y_eq = Y_EQ(x)  
    lambda_val = lambda_x(x, m, g_starS, g_star, Lambda)
    
    if x > 100:  # Arbitrary cutoff for equilibrium; modify as needed
        return np.array([0])
    
    result = -lambda_val * x**(-n-2) * (Y**2 - Y_eq**2)
    return np.array([result])
"""